Importando as bibliotecas e dependências necessárias:

In [1]:
# Bibliotecas necessárias
import os
import pandas as pd
import sqlite3
import time
import logging

/home/jvcampos/projects/etl-voos-anac/.venv/lib/python3.11/site-packages/numpy/_core/getlimits.py:551: UserWarning: Signature b'\x00\xd0\xcc\xcc\xcc\xcc\xcc\xcc\xfb\xbf\x00\x00\x00\x00\x00\x00' for <class 'numpy.longdouble'> does not match any known type: falling back to type probe function.
This warnings indicates broken support for the dtype!
  machar = _get_machar(dtype)


Carregando as bases de dados:

In [2]:
# Carregamento das bases de dados

# Base de dados voos ANAC
df_ocorrencias = pd.read_csv('../data/raw/V_OCORRENCIA_AMPLA.csv', sep=';', encoding='utf-8', skiprows=1)

# Base de dados municipios IBGE
df_municipios = pd.read_csv('../data/raw/municipios.csv', sep=';', encoding='latin1')

# Base de dados ocorrências CENIPA
df_cenipa = pd.read_excel('../data/raw/cenipa.xlsx')

In [3]:
# Configuração para exibir todos os dados e colunas
pd.set_option('display.max_columns', None)   # mostra todas as colunas
pd.set_option('display.max_colwidth', None)  # mostra todo o conteúdo da célula (mesmo que seja longo)

In [4]:
df_municipios.head()

,CÓDIGO DO MUNICÍPIO - TOM,CÓDIGO DO MUNICÍPIO - IBGE,MUNICÍPIO - TOM,MUNICÍPIO - IBGE,UF
0,1,1100106,GUAJARÁ-MIRIM,Guajará-Mirim,RO
1,2,1100379,ALTO ALEGRE DOS PARECIS,Alto Alegre dos Parecis,RO
2,3,1100205,PORTO VELHO,Porto Velho,RO
3,4,1100452,BURITIS,Buritis,RO
4,5,1100122,JI-PARANÁ,Ji-Paraná,RO


In [5]:
df_ocorrencias.head()

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,ICAO,Latitude,Longitude,Tipo_de_Aerodromo,Historico,Matricula,Categoria_da_Aeronave,Operador,Tipo_de_Ocorrencia,Fase_da_Operacao,Operacao,Danos_a_Aeronave,Aerodromo_de_Destino,Aerodromo_de_Origem,Lesoes_Fatais_Tripulantes,Lesoes_Fatais_Passageiros,Lesoes_Fatais_Terceiros,Lesoes_Graves_Tripulantes,Lesoes_Graves_Passageiros,Lesoes_Graves_Terceiros,Lesoes_Leves_Tripulantes,Lesoes_Leves_Passageiros,Lesoes_Leves_Terceiros,Ilesos_Tripulantes,Ilesos_Passageiros,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,25043,202019110,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,Incidente,2020-06-17,22:10,ALTO ALEGRE,RR,Norte,OUTROS,SJYG,"2,765","-62,2214",Privado,"A ORIGEM E DESTINO ERAM DESCONHECIDOS, BEM COMO A FINALIDADE DO VOO. DE ACORDO COM INFORMAÇÕES DO OPERADOR, POR VOLTA DAS 18H10MIN LOCAL O PILOTO TEVE PANE NO GPS DA AERONAVE E SIMULTANEAMENTE ENCONTROU CONDIÇÕES METEOROLÓGICAS ADVERSAS NA ROTA, REALIZANDO UM POUSO DE PRECAUÇÃO NO AERÓDROMO ALTO MUCAJAÍ ( SJYG ), ALTO ALEGRE, RR, ÁREA INDÍGENA YANOMAMI. APÓS O POUSO A AERONAVE FOI DANIFICADA E O PILOTO DETIDO PELOS INDÍGENAS. A AERONAVE NÃO TEVE DANOS DURANTE A REALIZAÇÃO DO POUSO DE PRECAUÇÃO. O TRIPULANTE SAIU ILESO.",PTCZC,TPP,CARAJÁS SERVIÇOS DE LOCAÇÃO LTDA EPF,OTHR,Indeterminada,Voo Privado,Nenhum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,182K,L1P,C182,1270.0,4.0,CESSNA AIRCRAFT,verdadeiro
1,25081,0940/2013,TAP AIR PORTUGAL,Incidente,2013-04-13,17:56,Brasília,DF,Centro-Oeste,COLISÃO NO SOLO,SBBR,"-15,8692","-47,9208",Público,"No dia 13 de abril de 2013 a aeronave A-330 da TAP, procedente da cidade de lisboa, pousou na pista 29L no aeroporto internacional de Brasília, quando ao iniciar o táxi para o pátio de estacionamento, foi orientado pelo Controle de Solo para prosseguir pelas TWY M, R e L4. Às 14:56 (horário local), ao taxiar pela TWY L4, a aeronave colidiu a ponta da asa esquerda contra uma torre de iluminação, provocando danos a winglet da referida asa. Todos os 239 passageiros e 11 tripulantes saíram ilesos.",CSTOF,Estrangeira,TAP AIR PORTUGAL,GCOL,Táxi,Voo Regular,Leve,SBBR,XXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso
2,25503,202023366,AZUL,Incidente,2020-10-02,19:02,CAMPINAS,SP,Sudeste,FOGO/FUMAÇA,SBKP,"-23,0069","-47,1344",Público,"A AERONAVE DECOLOU DO AERÓDROMO VIRACOPOS (SBKP), CAMPINAS, SP, COM DESTINO AO AERÓDROMO HERCÍLIO LUZ (SBFL), FLORIANÓPOLIS, SC, A FIM DE TRANSPORTAR PESSOAL, COM CINCO TRIPULANTES E CENTO E TRINTA E NOVE PASSAGEIROS A BORDO.DURANTE A SUBIDA, OCORREU A INDICAÇÃO DE 'DUE MID EBAY SMOKE'. A TRIPULAÇÃO EFETUOU OS PROCEDIMENTOS NO QRH E DECIDIU POR RETORNAR PARA O AERÓDROMO DE ORIGEM.FOI DECLARADA EMERGÊNCIA. O POUSO FOI REALIZADO NORMALMENTE E SEM INTERCORRÊNCIAS.A AERONAVE NÃO TEVE DANOS.OS CINCO TRIPULANTES E OS CENTO E TRINTA E NOVE PASSAGEIROS SAÍRAM ILESOS.",PSAEA,TPR,AZUL LINHAS AEREAS BRASILEIRAS S.A,F-NI,Subida,Voo Regular,Nenhum,SBFL,SBKP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,139.0,NaN,NaN,NaN,ERJ 190-400,L2J,E295,61500.0,143.0,EMBRAER,verdadeiro
3,25623,202021754,JOAO MAURICIO GOTTARDI LOPES,Incidente,2020-10-07,15:30,ARAÇATUBA,SP,Sudeste,CONTATO ANORMAL COM A PISTA,SBAU,"-21,1442","-50,4264",Público,"A AERONAVE DECOLOU DO AERÓDROMO GOVERNADOR JOSÉ RICHA (SBLO), LONDRINA, PR, COM DESTINO AO AERÓDROMO DE ARAÇATUBA (SBAU), ARAÇATUBA, SP, PARA UM VOO PRIVADO, COM UM TRIPULANTE E UM PASSAGEIRO A BORDO. NA FINAL PARA O POUSO, SEGUNDO RELATO DO PILOTO, HOUVE UMA RAJADA DE VENTO E AO TOCAR A PISTA HOUVE O DESPRENDIMENTO DA RODA DA BEQUILHA. A AERONAVE TEVE DANOS LEVES. O PILOTO E O PASSAGEIRO SAÍRAM ILESOS.",PTJBJ,TPP,JOAO MAURICIO GOTTARDI LOPES,ARC,Pouso,Voo Privado,Leve,SBAU,SBLO,NaN,NaN,NaN,NaN,N

In [6]:
df_cenipa.head()

,Link,Data,Matrícula,Classificação,Tipo,Localidade,UF,Aeródromo,Operação,Status
0,https://dedalo.sti.fab.mil.br/ocorrencia/87243,2025-06-17,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,CURITIBA,PR,SBBI,PRIVADA,FINALIZADA
1,https://dedalo.sti.fab.mil.br/ocorrencia/87235,2025-06-13,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,PACARAIMA,RR,SJML,TÁXI AÉREO,FINALIZADA
2,https://dedalo.sti.fab.mil.br/ocorrencia/87239,2025-06-13,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,SALVADOR,BA,SBSV,TÁXI AÉREO,FINALIZADA
3,https://dedalo.sti.fab.mil.br/ocorrencia/87238,2025-06-12,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,RIO DE JANEIRO,RJ,NCAD,TÁXI AÉREO,FINALIZADA
4,https://dedalo.sti.fab.mil.br/ocorrencia/87216,2025-06-10,****,INCIDENTE,FALHA OU MAU FUNCIONAMENTO DE SISTEMA / COMPONENTE,CUIABÁ,MT,SBCY,PRIVADA,FINALIZADA


## Etapa 2 - **Modificando Colunas**

**2.1** - Alterando os nomes das colunas:

In [7]:
df_ocorrencias = df_ocorrencias.rename(columns={
'Numero_da_Ocorrencia': 'NR_OCR', 
'Numero_da_Ficha': 'NR_FI', 
'Operador_Padronizado': 'OP_PDR', 
'Classificacao_da_Ocorrencia': 'CLAS_OCR', 
'Data_da_Ocorrencia':'DT_OCR', 
'Hora_da_Ocorrencia':'H_OCR', 
'Municipio':'MCP', 
'UF':'UF', 
'Regiao':'RG', 
'Descricao_do_Tipo':'DCRI_TP', 
'ICAO':'ICAO',
'Latitude':'LATD', 
'Longitude':'LONG', 
'Tipo_de_Aerodromo':'TP_ADRM', 
'Historico':'HIST', 
'Matricula':'MTCL', 
'Categoria_da_Aeronave':'CATG_ANV', 
'Operador':'OPOR', 
'Tipo_de_Ocorrencia':'TP_OCR', 
'Fase_da_Operacao':'FA_OP', 
'Operacao':'OPCAO', 
'Danos_a_Aeronave':'DAN_ANV', 
'Aerodromo_de_Destino':'ADRM_DSTN', 
'Aerodromo_de_Origem':'ADRM_ORGM', 
'Lesoes_Fatais_Tripulantes':'LS_FTL_TRPL',
'Lesoes_Fatais_Terceiros':'LS_FTL_TCOS',
'Lesoes_Fatais_Passageiros':'LS_FTL_PSGO',
'Lesoes_Graves_Tripulantes':'LS_GV_TRPL',
'Lesoes_Graves_Passageiros':'LS_GV_PSGO',
'Lesoes_Graves_Terceiros':'LS_GV_TCOS',
'Lesoes_Leves_Tripulantes':'LS_LV_TRPL',
'Lesoes_Leves_Passageiros':'LS_LV_PSGO',
'Lesoes_Leves_Terceiros':'LS_LV_TCOS',
'Ilesos_Tripulantes':'IL_TRPL',
'Ilesos_Passageiros':'IL_PSGO',
'Lesoes_Desconhecidas_Tripulantes':'LS_DESC_TRPL',
'Lesoes_Desconhecidas_Passageiros':'LS_DESC_PSGO',
'Lesoes_Desconhecidas_Terceiros':'LS_DESC_TCOS',
'Modelo':'MDL',
'CLS':'CLS',
'Tipo_ICAO':'TP_ICAO',
'PMD':'PMD',
'Numero_de_Assentos':'NR_ASTS',
'Nome_do_Fabricante':'NM_FBRC',
'PSSO':'PSSO',
})

**2.2** - Tipando os tipos de dados das colunas:

In [15]:

# Garante que todas as colunas sejam exibidas
pd.set_option('display.max_columns', None)

# Garante que todas as linhas sejam exibidas (ou ajuste para o número desejado)
pd.set_option('display.max_rows', None)

# Garante que o conteúdo das colunas não seja cortado
pd.set_option('display.max_colwidth', None)

# Exibe as primeiras linhas sem truncamento
linhas_nulas = df_ocorrencias[df_ocorrencias['NR_ASTS'].isna()]
linhas_nulas.head()  # ou apenas linhas_nulas.head() em notebooks


,NR_OCR,NR_FI,OP_PDR,CLAS_OCR,DT_OCR,H_OCR,MCP,UF,RG,DCRI_TP,ICAO,LATD,LONG,TP_ADRM,HIST,MTCL,CATG_ANV,OPOR,TP_OCR,FA_OP,OPCAO,DAN_ANV,ADRM_DSTN,ADRM_ORGM,LS_FTL_TRPL,LS_FTL_PSGO,LS_FTL_TCOS,LS_GV_TRPL,LS_GV_PSGO,LS_GV_TCOS,LS_LV_TRPL,LS_LV_PSGO,LS_LV_TCOS,IL_TRPL,IL_PSGO,LS_DESC_TRPL,LS_DESC_PSGO,LS_DESC_TCOS,MDL,CLS,TP_ICAO,PMD,NR_ASTS,NM_FBRC,PSSO
1,25081,0940/2013,TAP AIR PORTUGAL,Incidente,2013-04-13,17:56,Brasília,DF,Centro-Oeste,COLISÃO NO SOLO,SBBR,"-15,8692","-47,9208",Público,"No dia 13 de abril de 2013 a aeronave A-330 da TAP, procedente da cidade de lisboa, pousou na pista 29L no aeroporto internacional de Brasília, quando ao iniciar o táxi para o pátio de estacionamento, foi orientado pelo Controle de Solo para prosseguir pelas TWY M, R e L4. Às 14:56 (horário local), ao taxiar pela TWY L4, a aeronave colidiu a ponta da asa esquerda contra uma torre de iluminação, provocando danos a winglet da referida asa. Todos os 239 passageiros e 11 tripulantes saíram ilesos.",CSTOF,Estrangeira,TAP AIR PORTUGAL,GCOL,Táxi,Voo Regular,Leve,SBBR,XXXX,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,239.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso
21,5552,/2009,VIA AER,Acidente,2009-07-07,NaN,NaN,PR,Sul,FALHA OU MAU FUNCIONAMENTO DO MOTOR,NaN,NaN,NaN,NaN,NaN,N8540A,ESTRANGEIRA,VIA AER,SCF-PP,NaN,Voo Privado,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,falso
31,26044,202030691,NaN,Incidente,2020-11-07,09:39,BOITUVA,SP,Sudeste,COLISÃO COM OBSTÁCULO DURANTE A DECOLAGEM E POUSO,Fora de Aeródromo,NaN,NaN,-,"O balão decolou de localidade desconhecida, no município de Boituva, SP, a fim de realizar um voo local, com dois tripulantes e dez passageiros a bordo. Durante o pouso, uma rajada de vento levou o balão em direção a fios de alta tensão próximos ao local, vindo a ocorrer a colisão. O balão não sofreu danos. Todos a bordo saíram ilesos.",*****,Sem matrícula,NaN,CTOL,Pouso,Desconhecida,Nenhum,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,verdadeiro
54,27087,202180316,NaN,Acidente,2020-11-08,21:00,ARAPONGAS,PR,Sul,INDETERMINADO,Fora de Aeródromo,"-23,3953","-51,5383",-,"A aeronave decolou da fazenda Lageado, no município de Arapongas, PR, por volta das 21h00min (UTC), a fim de realizar voo local, com um piloto a bordo. Após a decolagem, a aeronave perdeu altura e colidiu contra o solo. A aeronave teve danos substanciais. O tripulante sofreu lesões fatais.",*****,PET,NaN,UNK,Indeterminada,Voo Experimental,Substancial,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso
60,25925,202039322,NaN,Incidente Grave,2020-03-01,23:00:00,CUIABÁ,MT,Centro-Oeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,SBCY,"15,65","-56,1175",Público,"A aeronave decolou do Aeroporto Cidade do Panamá Paitilla (MPMG), Panamá, com destino ao Aeródromo de Viracopos (SBKP), Campinas, SP, a fim de realizar voo privado, com dois pilotos e dois passageiros a bordo.Durante o voo de cruzeiro no FL 450 ocorreram múltiplos maus funcionamentos nos sistemas da aeronave, culminando com a falha em ambos os motores. Durante a descida, ao cruzar o FL 280, os pilotos conseguiram reacender os dois motores. Foi declarada situação de emergência. O pouso foi alternado para o Aeródromo Marechal Rondon (SBCY), Cuiabá, MT. A aeronave pousou normalmente.Não houve danos à aeronave.Todos a bordo saíram ilesos.",N36DA,Estrangeira,NaN,SCF-PP,Cruzeiro,Voo Privado,Nenhum,SBKP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,falso


In [13]:
df_ocorrencias['LATD'].isna().sum()
df_ocorrencias['LATD'].head()

0     2.7650
1   -15.8692
2   -23.0069
3   -21.1442
4   -23.5100
Name: LATD, dtype: float64

In [16]:
df_ocorrencias['DT_OCR'] = pd.to_datetime(df_ocorrencias['DT_OCR'], errors='coerce').dt.normalize()

df_ocorrencias['LS_FTL_TRPL'] = pd.to_numeric(df_ocorrencias['LS_FTL_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_FTL_TCOS'] = pd.to_numeric(df_ocorrencias['LS_FTL_TCOS'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_FTL_PSGO'] = pd.to_numeric(df_ocorrencias['LS_FTL_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_GV_TRPL'] = pd.to_numeric(df_ocorrencias['LS_GV_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_GV_PSGO'] = pd.to_numeric(df_ocorrencias['LS_GV_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_GV_TCOS'] = pd.to_numeric(df_ocorrencias['LS_GV_TCOS'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_LV_TRPL'] = pd.to_numeric(df_ocorrencias['LS_LV_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_LV_PSGO'] = pd.to_numeric(df_ocorrencias['LS_LV_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_LV_TCOS'] = pd.to_numeric(df_ocorrencias['LS_LV_TCOS'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['IL_TRPL'] = pd.to_numeric(df_ocorrencias['IL_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['IL_PSGO'] = pd.to_numeric(df_ocorrencias['IL_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_DESC_TRPL'] = pd.to_numeric(df_ocorrencias['LS_DESC_TRPL'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_DESC_PSGO'] = pd.to_numeric(df_ocorrencias['LS_DESC_PSGO'], errors='coerce').astype('Int64').fillna(0).astype(int)
df_ocorrencias['LS_DESC_TCOS'] = pd.to_numeric(df_ocorrencias['LS_DESC_TCOS'], errors='coerce').astype('Int64').fillna(0).astype(int)

# Colunas Latitude e Longitude = LATD e LONG
## Trocando vírgulas por ponto para realizar a tipagem
df_ocorrencias['LONG'] = (
    df_ocorrencias['LONG']
    .astype(str)
    .str.replace(',', '.', regex=False)
)

df_ocorrencias['LONG'] = pd.to_numeric(df_ocorrencias['LONG'], errors='coerce')

df_ocorrencias['LATD'] = (
    df_ocorrencias['LATD']
    .astype(str)                    # Garante que tudo é string
    .str.replace(',', '.', regex=False)  # Troca vírgula por ponto
)

df_ocorrencias['LATD'] = pd.to_numeric(df_ocorrencias['LATD'], errors='coerce')


# df_ocorrencias['NR_ASTS'] = pd.to_numeric(df_ocorrencias['NR_ASTS'], errors='coerce').astype('Int64').fillna(0).astype(int)

df_ocorrencias['PSSO'] = df_ocorrencias['PSSO'].apply(
    lambda x: str(x).strip().lower() == 'verdadeiro'
)

In [17]:
df_ocorrencias.dtypes

NR_OCR                   int64
NR_FI                   object
OP_PDR                  object
CLAS_OCR                object
DT_OCR          datetime64[ns]
H_OCR                   object
MCP                     object
UF                      object
RG                      object
DCRI_TP                 object
ICAO                    object
LATD                   float64
LONG                   float64
TP_ADRM                 object
HIST                    object
MTCL                    object
CATG_ANV                object
OPOR                    object
TP_OCR                  object
FA_OP                   object
OPCAO                   object
DAN_ANV                 object
ADRM_DSTN               object
ADRM_ORGM               object
LS_FTL_TRPL              int64
LS_FTL_PSGO              int64
LS_FTL_TCOS              int64
LS_GV_TRPL               int64
LS_GV_PSGO               int64
LS_GV_TCOS               int64
LS_LV_TRPL               int64
LS_LV_PSGO               int64
LS_LV_TC

## 3 - Preenchendo valores ausentes

Colunas Modelo, Nome do Fabricante e Número de Assentos = MDL, NM_FBRC, NR_ASTS


Valor PMD médio para Balões = 2400.00

CLS não identificado = Desconhecido


In [47]:
# Encontrado na WEB - fonte: https://www.correiobraziliense.com.br/app/noticia/cidades/2013/04/13/interna_cidadesdf%2C360268/aviao-da-tap-bate-em-torre-de-iluminacao-do-aeroporto-jk-nao-ha-feridos.shtml
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'MDL'] = 'Airbus A330-223'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'NM_FBRC'] = 'Airbus'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'NR_ASTS'] = 268
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'TP_ICAO'] = 'Desconhecido'
# Fonte do PMD:  https://skyteamvirtual.org/fleet/models/delta-air-lines-airbus-a330-223?
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 25081, 'PMD'] = 242000.00


# Encontrado na base de dados do CENIPA: https://dedalo.sti.fab.mil.br/ocorrencia/79664
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'MDL'] = 'Balão'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'NM_FBRC'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'NR_ASTS'] = 12
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'PMD'] = 0
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'CLS'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'PMD'] = 2400.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'LATD'] = -23.2854
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'LONG'] = -47.674
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'OPOR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'OP_PDR'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'ADRM_DSTN'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'ADRM_ORGM'] = 'FAER'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 26044, 'TP_ICAO'] = 'Desconhecido'


# Encontrado na WEB: https://www.gazetadopovo.com.br/vida-e-cidadania/aeronave-de-pequeno-porte-cai-em-curitiba-bnfeefp83hvbfbs5a6zzp1kjy/
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'MDL'] = 'Bonanza A35'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'NM_FBRC'] = 'Beechcraft'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'NR_ASTS'] = 4
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'MCP'] = 'CURITIBA'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'H_OCR'] = '16:00'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'TP_ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'ICAO'] = 'Desconhecido'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'LATD'] = -25.55982683577607
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'LONG'] = -49.23159109274909
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'HIST'] = 'Uma aeronave de pequeno porte caiu perto do zoológico de Curitiba na tarde de terça-feira (7), após uma pane no motor. O avião, modelo Bonanza A35, vinha de Londrina com destino ao Aeroporto Afonso Pena. O piloto conseguiu fazer um pouso forçado em área aberta no Parque Iguaçu. Ninguém ficou ferido e houve apenas danos materiais. Os dois ocupantes não precisaram de atendimento médico e não deram declarações à imprensa.'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'FA_OP'] = 'Indeterminada'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'ADRM_DSTN'] = 'Aeroporto Afonso Pena'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'ADRM_ORGM'] = 'Londrina'
# Fonte do PMD: https://www.bonanza.org/aircraft-index/browse-specific-models/bonanza/bonanza-a35/
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'PMD'] = 2400.00
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'TP_ADRM'] = 'Público'
df_ocorrencias.loc[df_ocorrencias['NR_OCR'] == 5552, 'DAN_ANV'] = 'Leve'



In [45]:
# df_ocorrencias[df_ocorrencias['NR_ASTS'].isnull()]
df_ocorrencias[df_ocorrencias['NR_OCR'] == 5552]


,NR_OCR,NR_FI,OP_PDR,CLAS_OCR,DT_OCR,H_OCR,MCP,UF,RG,DCRI_TP,ICAO,LATD,LONG,TP_ADRM,HIST,MTCL,CATG_ANV,OPOR,TP_OCR,FA_OP,OPCAO,DAN_ANV,ADRM_DSTN,ADRM_ORGM,LS_FTL_TRPL,LS_FTL_PSGO,LS_FTL_TCOS,LS_GV_TRPL,LS_GV_PSGO,LS_GV_TCOS,LS_LV_TRPL,LS_LV_PSGO,LS_LV_TCOS,IL_TRPL,IL_PSGO,LS_DESC_TRPL,LS_DESC_PSGO,LS_DESC_TCOS,MDL,CLS,TP_ICAO,PMD,NR_ASTS,NM_FBRC,PSSO
21,5552,/2009,VIA AER,Acidente,2009-07-07,16:00,CURITIBA,PR,Sul,FALHA OU MAU FUNCIONAMENTO DO MOTOR,Desconhecido,-25.559827,-49.231591,Público,"Uma aeronave de pequeno porte caiu perto do zoológico de Curitiba na tarde de terça-feira (7), após uma pane no motor. O avião, modelo Bonanza A35, vinha de Londrina com destino ao Aeroporto Afonso Pena. O piloto conseguiu fazer um pouso forçado em área aberta no Parque Iguaçu. Ninguém ficou ferido e houve apenas danos materiais. Os dois ocupantes não precisaram de atendimento médico e não deram declarações à imprensa.",N8540A,ESTRANGEIRA,VIA AER,SCF-PP,Indeterminada,Voo Privado,NaN,Aeroporto Afonso Pena,Londrina,0,0,0,0,0,0,0,0,0,1,1,0,0,0,Bonanza A35,NaN,Desconhecido,2400.0,4,Beechcraft,False


## df_ocorrencias['DT_OCR'].head()

In [13]:
sorted(df_ocorrencias["MCP"].dropna().unique())

['\tCAMPO GRANDE',
 '\tCAMPO GRANDE ',
 '\tCAXIAS DO SUL',
 '\tFERNANDO DE NORONHA ',
 '\tITAITUBA',
 '\tPORTO ALEGRE',
 '\tPORTO ALEGRE ',
 '\tRIO DE JANEIRO',
 '\tRIO DE JANEIRO ',
 '\tSANTANA DO ARAGUAIA',
 '\tSÃO GONÇALO DO AMARANTE',
 '\tSÃO JOSÉ DOS PINHAIS',
 '\tSÃO PAULO',
 '\tSÃO PAULO ',
 '\tVÁRZEA GRANDE ',
 '  RIO DE JANEIRO ',
 ' BELO HORIZONTE',
 ' CATANDUVA ',
 ' Cidade do México, México',
 ' ESTADOS UNIDOS',
 ' FORTALEZA',
 ' Longitude Histórico',
 ' RIO DE JANEIRO',
 ' SÃO FÉLIX DO ARAGUAIA',
 ' SÃO JOSÉ DO RIO PRETO',
 ' SÃO PAULO ',
 ' VILA VELHA',
 '12:54',
 '22:48:36-S',
 '23:25:48-S',
 'ABADIA DE GOIÁS',
 'ACARÁ',
 'ACREÚNA',
 'AEROPORTO DE GUARULHOS',
 'AEROPORTO DE SÃO JOSÉ DOS CAMPOS',
 'AFONSO CLÁUDIO',
 'AFUÁ',
 'AGUA BOA',
 'AGUASCALIENTES, MÉXICO',
 'AGUAÍ',
 'AGUDO',
 'AGUDOS',
 'ALCÂNTARA',
 'ALEGRETE',
 'ALEGRETE ',
 'ALFENAS',
 'ALFREDO CHAVES',
 'ALIANÇA DOS TOCANTINS',
 'ALMEIRIM',
 'ALMIRANTE TAMANDARÉ',
 'ALPESTRE ',
 'ALTA FLORESTA',
 "ALTA FLOREST

In [14]:
df_ocorrencias["MCP"] = df_ocorrencias["MCP"].str.strip()
sorted(df_ocorrencias["MCP"].dropna().unique())

['12:54',
 '22:48:36-S',
 '23:25:48-S',
 'ABADIA DE GOIÁS',
 'ACARÁ',
 'ACREÚNA',
 'AEROPORTO DE GUARULHOS',
 'AEROPORTO DE SÃO JOSÉ DOS CAMPOS',
 'AFONSO CLÁUDIO',
 'AFUÁ',
 'AGUA BOA',
 'AGUASCALIENTES, MÉXICO',
 'AGUAÍ',
 'AGUDO',
 'AGUDOS',
 'ALCÂNTARA',
 'ALEGRETE',
 'ALFENAS',
 'ALFREDO CHAVES',
 'ALIANÇA DOS TOCANTINS',
 'ALMEIRIM',
 'ALMIRANTE TAMANDARÉ',
 'ALPESTRE',
 'ALTA FLORESTA',
 "ALTA FLORESTA D'OESTE",
 'ALTAIR',
 'ALTAMIRA',
 'ALTO ALEGRE',
 'ALTO ALEGRE DO PINDARÉ',
 'ALTO ARAGUAIA',
 'ALTO GARÇAS',
 'ALTO HORIZONTE',
 'ALTO PARAGUAI',
 'ALTO PARANÁ',
 'ALTO PARAÍSO',
 'ALTO PARAÍSO DE GOIÁS',
 'ALTO PIQUIRI',
 'ALTO SANTO',
 'ALTÔNIA',
 'AMAJARI',
 'AMAPORÃ',
 'AMAPÁ',
 'AMERICANA',
 'AMERICANO DO BRASIL',
 'ANAJÁS',
 'ANAMÃ',
 'ANANINDEUA',
 'ANAPU',
 'ANAPÓLIS',
 'ANAURILÂNDIA',
 'ANDRADINA',
 'ANGRA DOS REIS',
 'ANÁPOLIS',
 'APARECIDA DE GOIÂNIA',
 'APUCARANA',
 'APUÍ',
 'AQUIDAUANA',
 'AQUIRAZ',
 'ARACAJU',
 'ARACAJÚ',
 'ARACATI',
 'ARACRUZ',
 'ARAGARÇAS',
 'ARA

In [15]:
df_ocorrencias["MCP"] = df_ocorrencias["MCP"].str.strip()
num_distintos = df_ocorrencias["MCP"].dropna().nunique()
print(f"Número de municípios distintos: {num_distintos}")


Número de municípios distintos: 1439


In [16]:
df_ocorrencias["MCP"].isnull().sum()  # Conta só os nulos da coluna "Municipio"

np.int64(3698)

In [17]:

#O que fazer com esses dados?
df_ocorrencias[
    df_ocorrencias["MCP"].isnull() &
    df_ocorrencias["RG"].isnull() &
    df_ocorrencias["UF"].isnull() &
    df_ocorrencias["LATD"].isnull() &
    df_ocorrencias["LONG"].isnull()
]

,NR_OCR,NR_FI,OP_PDR,CLAS_OCR,DT_OCR,H_OCR,MCP,UF,RG,DCRI_TP,ICAO,LATD,LONG,TP_ADRM,HIST,MTCL,CATG_ANV,OPOR,TP_OCR,FA_OP,OPCAO,DAN_ANV,ADRM_DSTN,ADRM_ORGM,LS_FTL_TRPL,LS_FTL_PSGO,LS_FTL_TCOS,LS_GV_TRPL,LS_GV_PSGO,LS_GV_TCOS,LS_LV_TRPL,LS_LV_PSGO,LS_LV_TCOS,IL_TRPL,IL_PSGO,LS_DESC_TRPL,LS_DESC_PSGO,LS_DESC_TCOS,MDL,CLS,TP_ICAO,PMD,NR_ASTS,NM_FBRC,PSSO
7963,25805,202067408,AMERICAN AIRLINES,Acidente,2020-10-24,15:53,NaN,NaN,NaN,EXCURSÃO DE PISTA,MYGF,NaN,NaN,NaN,Aircraft landed on RWY 24 and shortly after making contact with the runway veered off of the runway resulting in a runway excursion. There were no injuries reported.,N674RJ,Estrangeira,AMERICAN AIRLINES,RE,Corrida após pouso,Voo Regular,NaN,MYGF,KMIA,0,0,0,0,0,0,0,0,0,2,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False
11271,40149,202318593475,TAM,Incidente,2023-10-29,17:45:00,NaN,NaN,NaN,COLISÃO COM AVE,Fora de Aeródromo,NaN,NaN,-,Houve colisão com ave.,PRMHQ,TPR,TAM LINHAS AEREAS S.A,BIRD,Indeterminada,Voo Regular,Leve,SBSP,SBUL,0,0,0,0,0,0,0,0,0,0,0,0,0,0,A320-214,L2J,A320,77000.0,189.0,AIRBUS S.A.S.,True
11457,36986,2023626632,CARGOLUX AIRLINES INTERNATIONAL S.A.,Incidente Grave,2018-01-10,03:10:00,NaN,NaN,NaN,FALHA OU MAU FUNCIONAMENTO DO MOTOR,Fora de Aeródromo,NaN,NaN,-,"During cruise between WPT 0524N and KODOS, at time 03:10Z the crew experienced an engine malfunction. The first indication was a smell/fumes in the Flight Deck. The crew don the crew oxygen masks followed by a smoke alarm in the R/H crew rest bunk. In mean time smoke was present in the entire crew rest area. thereafter the crew received an engine vibration indication and the engine #1EGT increased to 900 degree Celsius. The crew performed the required non-normal checklist. the smoke immediately seized while observing the engine #1 oil quantily being at zero. The crew assumed the source of the smoke was the oil leak on the failed engine#1. With the engine secured and no more smoke present, the crew decided to divert to Recife for a safe landing. the crew declared a Pan Pan situation with ATC and landed safely. The aircraft is presently AOG.",LXUCV,Estrangeira,CARGOLUX AIRLINES INTERNATIONAL S.A.,SCF-PP,Cruzeiro,Voo Regular,NaN,SBKP,ELLX,0,0,0,0,0,0,0,0,0,3,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,False


In [18]:
# Converta para sets, eliminando nulos e garantindo que tudo seja string
todos_municipios = set(df_municipios['MUNICÍPIO - TOM'].dropna().astype(str).str.strip().str.upper())
municipios_incidentes = set(df_ocorrencias['MCP'].dropna().astype(str).str.strip().str.upper())

# Encontre os valores exclusivos de cada conjunto
municipios_so_em_um = (todos_municipios ^ municipios_incidentes)

# Ordenar os que estão apenas em df_ocorrencias
municipios_somente_em_df_incidentes = sorted(municipios_incidentes - todos_municipios)

print(municipios_somente_em_df_incidentes)


['12:54', '22:48:36-S', '23:25:48-S', 'AEROPORTO DE GUARULHOS', 'AEROPORTO DE KIMBERLEY', 'AEROPORTO DE SÃO JOSÉ DOS CAMPOS', 'AGUA BOA', 'AGUASCALIENTES, MÉXICO', 'ALIANÇA DOS TOCANTINS', 'ANAPÓLIS', 'ARACAJÚ', 'ARGENTINA', 'ARIPUAÑA', 'ASSIS CHATEAUBRIAN', 'BAIRRO MULTIRÃO I', 'BALNEARIO CAMBORIU', 'BARRA DO GARCAS', 'BASE AÉREA DE SANTOS', 'BELA VISTA DO PARAISO', 'BELO HORIZONE', 'BIRITIBA MIRIM', 'BLUMENAL', 'BOA VISTA DOS RAMOS', 'BOM JESUS DE GOIÁS', 'BONOPÓLIS', 'BRAISLÂNDIA', 'BRASILIA', 'BRASÌLIA', 'BRASÍLIA,', 'BRASÍLIADF', 'BUENOS AIRES, ARGENTINA', 'CAIXIAS DO SUL', 'CAMOINAS', 'CAMPINA GRANDE |', 'CAMPO DE MARTE', 'CAMPO DOS GOYTACAZES', 'CAMPO GRANDEMS', 'CAMPO GRNADE', 'CAMPO MORÃO', 'CAMPOS', 'CAMPOS DOS GOITACAZES', 'CAMPOS DOS GOYTACAS', 'CAMPOS DOS GOYTACASES', 'CAMPPINAS', 'CANCUN, MÉXICO', 'CANCÚN, MÉXICO', 'CAPE TOWN', 'CAPÃO DO CANOA', 'CARLÂNDIA', 'CASIMIRO E ABREU', 'CHAPADA DOS GUIMA', 'CHITOSE, JAPÃO', 'CIDADE DE UNA', 'CIDADE DO MÉXICO, MÉXICO', 'COLIDER', 

In [19]:
pd.set_option('display.max_columns', None)   # mostra todas as colunas
pd.set_option('display.max_colwidth', None)  # mostra todo o conteúdo da célula (mesmo que seja longo)

In [20]:
# Excluindo valores
df_ocorrencias = df_ocorrencias[df_ocorrencias['MCP'].isin([
    'AGUASCALIENTES, MÉXICO',
    'ARGENTINA',
    'CANCUN, MÉXICO',
    'CANCÚN, MÉXICO',
    'CAPE TOWN',
    'CHITOSE, JAPÃO',
    'GUANAJUATO',
    'CIDADE DO MÉXICO, MÉXICO',
    'LISBOA',
    'MAR DEL PLATA',
    'MISSIONES',
    'ESTADOS UNIDOS',
    'FORA DO BRASIL',
    'GUANAJUATO, MÉXICO',
    'KUALA LUMPUR, MALÁSIA',
    'LISBOA/POTUGAL',
    'LISBOA ,PORTUGAL',
    'MADRID',
    'MAR DEL PLATA, ARGENTINA',
    'MIAMI',
    'MISSIONES,ARG',
    'MONTEVIDEO',
    'MORELOS, MÉXICO',
    'PARIS',
    'PEDRO JUAN CABALLERO',
    'SENEGAL, ÁFRICA',
    'SILAO, GUANAJUATO, MÉXICO',
    'SOMÁLIA, ÁFRICA'
])]

df_ocorrencias = df_ocorrencias.reset_index(drop=True) # Resetar o índice


df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '202437239132', 'Municipio'] = 'Salvador'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '202346987620', 'Municipio'] = 'Petrópolis'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '202389664945', 'Municipio'] = 'Santa Isabel'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '201107163514591', 'Municipio'] = 'Tabatinga'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '201705041722198/2017', 'Municipio'] = 'Belo Horizonte'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == 'PT-VEB 11MAI2017 17H00 SERIPA3', 'Municipio'] = 'Uberlândia'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == 'PU-PPY 28ABR2019 15H30 SERIPA1', 'Municipio'] = 'Peixoto de Azevedo'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '202312268204', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '202312268204', 'UF'] = 'SP'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '202312268204', 'Regiao'] = 'Sudeste'
df_ocorrencias.loc[df_ocorrencias['NR_FI'] == '2022445310', 'Municipio'] = 'Rondolândia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'AEROPORTO DE GUARULHOS', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'AEROPORTO DE SÃO JOSÉ DOS CAMPOS', 'Municipio'] = 'São José dos Campos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'AGUA BOA', 'Municipio'] = 'Água boa'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ÁGUAS CLARAS', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ALIANÇA DOS TOCANTINS', 'Municipio'] = 'Aliança do Tocantins'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ANAPÓLIS', 'Municipio'] = 'Anápolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ARACAJÚ', 'Municipio'] = 'Aracaju'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ARIPUAÑA', 'Municipio'] = 'Aripuanã'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ASSIS CHATEAUBRIAN', 'Municipio'] = 'Assis Chateaubriand'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BALNEARIO CAMBORIU', 'Municipio'] = 'Balneário Camboriú'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BARRA DO GARCAS', 'Municipio'] = 'Barra do Garças'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BASE AÉREA DE SANTOS', 'Municipio'] = 'Santos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BELA VISTA DO PARAISO', 'Municipio'] = 'Bela Vista do Paraíso'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BELO HORIZONE', 'Municipio'] = 'Belo Horizonte'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BIRITIBA MIRIM', 'Municipio'] = 'Biritiba-Mirim'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BLUMENAL', 'Municipio'] = 'Blumenau'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BOA VISTA DOS RAMOS', 'Municipio'] = 'Boa Vista do Ramos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BOM JESUS DE GOIÁS', 'Municipio'] = 'Bom Jesus'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BONOPÓLIS', 'Municipio'] = 'Bonópolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRAISLÂNDIA', 'Municipio'] = 'Brasilãndia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRASILIA', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRASÌLIA', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRASÍLIA,', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'BRASÍLIADF', 'Municipio'] = 'Brasília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAIXIAS DO SUL', 'Municipio'] = 'Caxias do Sul'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMOINAS', 'Municipio'] = 'Campinas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPINA GRANDE |', 'Municipio'] = 'Campina Grande'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO DE MARTE'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO DOS GOYTACAZES', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO GRANDEMS', 'Municipio'] = 'Campo Grande'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO GRNADE', 'Municipio'] = 'Campo Grande'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPO MORÃO', 'Municipio'] = 'Campo Mourão'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPPINAS', 'Municipio'] = 'Campinas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPOS', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPOS DOS GOITACAZES', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPOS DOS GOYTACAS', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAMPOS DOS GOYTACASES', 'Municipio'] = 'Campos dos Goytacazes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CAPÃO DO CANOA', 'Municipio'] = 'Capão da Canoa'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CARLÂNDIA', 'Municipio'] = 'Indiara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CASIMIRO E ABREU', 'Municipio'] = 'Casimirio de Abreu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CHAPADA DOS GUIMA', 'Municipio'] = 'Chapada dos Guimarães'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CIDADE DE UNA', 'Municipio'] = 'Una'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'COLIDER', 'Municipio'] = 'Colíder'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CONSELHEIRO LAFAYETE', 'Municipio'] = 'Conselheiro Lafaiete'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CUARI', 'Municipio'] = 'Coari'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CUIABA', 'Municipio'] = 'Cuiabá'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CURITBA', 'Municipio'] = 'Curitiba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CURITUBA', 'Municipio'] = 'Curitiba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'CURUTIBA', 'Municipio'] = 'Curitiba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ENTRE-IJUÍS', 'Municipio'] = 'Entre Ijuis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ESPINGARDA', 'Municipio'] = 'Eliseu Martins'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FLOARIANÓPOLIS', 'Municipio'] = 'Florianópolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FOZ DO IGUAÇ', 'Municipio'] = 'Foz do Iguaçu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GAURULHOS', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GAUÍRA', 'Municipio'] = 'Guaíra'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GOIANIA', 'Municipio'] = 'Goiânia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GUARILHOS', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GUARULHOSS', 'Municipio'] = 'Guarulhos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA BOA VISTA, QUIRINÓPOLIS', 'Municipio'] = 'Quirinópolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA FORTALEZA DE SANTA TEREZINHA', 'Municipio'] = 'Santa Terezinha'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA FLORESTA, BRASNORTE', 'Municipio'] = 'Brasnorte'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA INHUMAS DO CHAPADÃO', 'Municipio'] = 'Uberaba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA INHUMAS DO CHAPADÃO, MG', 'Municipio'] = 'Uberaba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA NOVA AURORA, CAROLINA', 'Municipio'] = 'Carolina'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA NOVA ESPERANÇA', 'Municipio'] = 'Balsas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA NOVA ESPERANÇA, BALSAS', 'Municipio'] = 'Balsas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA SANTA BÁRBARA', 'Municipio'] = 'Bom Jesus'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA SANTA BÁRBARA, BOM JESUS DE GOIÁS', 'Municipio'] = 'Bom Jesus'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA SANTO INACIO', 'Municipio'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA SANTO INÁCIO – RS', 'Municipio'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA TRES FIGUEIRAS', 'Municipio'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA TRÊS FIGUEIRAS', 'Municipio'] = 'Itaqui'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA UBIRATÃ', 'Municipio'] = 'Primavera do Leste'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'FAZENDA VIGOLO', 'Municipio'] = 'Palmas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'GRAMADINHO', 'Municipio'] = 'Seara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'HANGAR LÍDER AVIAÇÃO', 'Municipio'] = 'Belém'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ILHÈUS'] = 'Ilhéus'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'IGARASSÚ'] = 'Igarassu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ITANHAEM'] = 'Itanhaém'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ITAPORANGA D’ AJUDA'] = "Itaporanga D'Ajuda"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JACAREPAGUÁ'] = "Rio de Janeiro"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JERICOACOARA'] = "Jijoca de Jericoacoara"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JI-PARANÃ'] = "Ji-Paraná"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JUINA'] = "Juína"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'JUNDIAI'] = "Jundiaí"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LENÇOIS'] = "Lençóis"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LENÇOIS PAULISTAS'] = "Lençóis Paulista"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LENÇOIS PAULISTA'] = "Lençóis Paulista"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LENÇÓIS PAULISTAS'] = "Lençóis Paulista"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LONDRINA, PR'] = "Londrina"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LUÍS EDUARDO MAGAL', 'Municipio'] = 'Luís Eduardo Magalhães'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'LUIS EDUARDO MAGALHÃES', 'Municipio'] = 'Luís Eduardo Magalhães'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MANUEL URBANO', 'Municipio'] = 'Manoel Urbano'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MARICA', 'Municipio'] = 'Maricá'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MARILIA', 'Municipio'] = 'Marília'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MAÇAMBARÁ', 'Municipio'] = 'Maçambara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MIGRANTINÓPOLIS', 'Municipio'] = 'Novo Horizonte do Oeste'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MIRASSOL D´OESTE', 'Municipio'] = "Mirassol D'Oeste"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MOGI GUAÇU', 'Municipio'] = "Mogi-Guaçu"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MOGI-MIRIM', 'Municipio'] = "Mogi Mirim"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'MONTES CLAROS,', 'Municipio'] = "Montes Claros"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NAVIRAI', 'Municipio'] = "Naviraí"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NITEROI', 'Municipio'] = "Niterói"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NOVA ARIPUANÃ', 'Municipio'] = 'Novo Aripuanã'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NOVA IGUAÇU', 'Municipio'] = 'Nova Iguaçu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NOVA IGUAÇÚ', 'Municipio'] = 'Nova Iguaçu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'NOVA MARINGA', 'Municipio'] = 'Nova Maringá'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PALIMIU', 'Municipio'] = 'Barcelos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PARATININGA', 'Municipio'] = 'Paranatinga'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PARATY', 'Municipio'] = 'Parati'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PARAUABEBAS', 'Municipio'] = 'Parauapebas'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PEDRA PRETA,', 'Municipio'] = 'Pedra Preta'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PONTES DE LACERDA', 'Municipio'] = 'Pontes e Lacerda'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PORO VELHO', 'Municipio'] = 'Porto Velho'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PORTO ESPERIDÃO', 'Municipio'] = 'Porto Esperidião '
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PRAIA DE ENSEADA - SÃO FRANCISCO DO SUL', 'Municipio'] = 'São Francisco do Sul'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'QUERENCIA DO NORTE', 'Municipio'] = 'Querência do Norte'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PORTO ALEGRE', 'Municipio'] = 'Porto Alegre'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PORTO ALEGRE |', 'Municipio'] = 'Porto Alegre'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'PIRAI DO SUL', 'Municipio'] = 'Piraí do Sul'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RECIFE', 'Municipio'] = 'Recife'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RECUFE', 'Municipio'] = 'Recife'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIO DE JANEIRO', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIACHÂO DAS NEVES', 'Municipio'] = 'Riachão das Neves'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIO DE JANERIO', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIO DE JANEIRO ', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RIO DE  JANEIRO', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'RORAINOPOLIS', 'Municipio'] = 'Rio de Janeiro'
df_ocorrencias.loc[df_ocorrencias['MCP'] == "SANT'ANA DO LIVRAMENTO", 'Municipio'] = 'Santana do Livramento'
df_ocorrencias.loc[df_ocorrencias['MCP'] == "SANTA CRUZ DAS PALMAS", 'Municipio'] = 'Santa Cruz das Palmeiras'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTANA DO PARAISO', 'Municipio'] = 'Santana do Paraíso'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTANA DO PARNAÍBA', 'Municipio'] = 'Santana de Parnaíba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTAREM', 'Municipio'] = 'Santarém'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTA TEREZINHA DE JESUS', 'Municipio'] = 'Santa Terezinha de Itaipu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTA TEREZINHA DO ITAIPU', 'Municipio'] = 'Santa Terezinha de Itaipu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTA VITÓRIA DO PAMAR', 'Municipio'] = 'Santa Vitória Do Palmar'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTO ANTONIO DO LEVERGER', 'Municipio'] = 'Santo Antônio do Leverger'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTO ANTÔNIO DO LEVERGER', 'Municipio'] = 'Santo Antônio do Leverger'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO XINGÚ', 'Municipio'] = 'São Felix do Xingu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO XINGÚ', 'Municipio'] = 'São Felix do Xingu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSE DOS CAMPOS', 'Municipio'] = 'São José dos Campos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSE DO RIO PRETO', 'Municipio'] = 'São José do Rio Preto'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO LUIZ', 'Municipio'] = 'São Luís'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO LUIZ', 'Municipio'] = 'São Luís'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTAÉM', 'Municipio'] = 'Santarém'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SLAVADOR', 'Municipio'] = 'Salvador'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃ PAULO', 'Municipio'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO ARAGUAIA', 'Municipio'] = 'São Félix do Araguaia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO XINGU', 'Municipio'] = 'São Félix do Xingu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO FELIX DO XINGÚ', 'Municipio'] = 'São Félix do Xingu'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SAO PAULO', 'Municipio'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOÃO DEL REY', 'Municipio'] = 'São João del Rei'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSE DOS PINHARES', 'Municipio'] = 'São José dos Pinhais'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSÉ DOS PINHARES', 'Municipio'] = 'São José dos Pinhais'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSÉ DO PINHAL', 'Municipio'] = 'São José dos Pinhais'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO JOSÉ DOS CAMOPOS', 'Municipio'] = 'São José dos Campos'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SÃO LUIS', 'Municipio'] = 'São Luís'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SERRRANÓPOLIS', 'Municipio'] = 'Serranópolis'
df_ocorrencias.loc[df_ocorrencias['MCP'] == "SÍTIO SÃO JOSÉ, ZONA RURAL DE GLÓRIA D'OESTE", 'Municipio'] = "Glória D'Oeste"
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TANGARÁ DAQ SERRA', 'Municipio'] = 'Tangaré da Serra'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TELEMACO BORBA', 'Municipio'] = 'Telêmaco Borba'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TIÊTE', 'Municipio'] = 'São Paulo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TRAJANO DE MORAIS', 'Municipio'] = 'Trajano de Moraes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'TUPANCI', 'Municipio'] = 'São Sepé'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'URBERLÂNDIA', 'Municipio'] = 'Uberlândia'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'VARZEA GRANDE', 'Municipio'] = 'Várza Grande'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'VERISSÍMO', 'Municipio'] = 'Veríssimo'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'VILA BELA DA SANTÍSSIMA', 'Municipio'] = 'Vila Bela da Santíssima Trindade'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'VITORIA DA CONQUISTA', 'Municipio'] = 'Vitória da Conquista'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'XINGUARÁ', 'Municipio'] = 'Xinguara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'YARA', 'Municipio'] = 'Bandeirantes'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'XINGUARÁ', 'Municipio'] = 'Xinguara'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ÉRICO - AMAJARI', 'Municipio'] = 'Amajari'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'ÁREA RURAL', 'Municipio'] = 'Palestina'
df_ocorrencias.loc[df_ocorrencias['MCP'] == 'SANTO ANTÔNIO DE LEVERGER', 'Municipio'] = 'Santo Antônio do Leverger'

In [21]:
df_ocorrencias[df_ocorrencias['MCP'].isnull()]

,NR_OCR,NR_FI,OP_PDR,CLAS_OCR,DT_OCR,H_OCR,MCP,UF,RG,DCRI_TP,ICAO,LATD,LONG,TP_ADRM,HIST,MTCL,CATG_ANV,OPOR,TP_OCR,FA_OP,OPCAO,DAN_ANV,ADRM_DSTN,ADRM_ORGM,LS_FTL_TRPL,LS_FTL_PSGO,LS_FTL_TCOS,LS_GV_TRPL,LS_GV_PSGO,LS_GV_TCOS,LS_LV_TRPL,LS_LV_PSGO,LS_LV_TCOS,IL_TRPL,IL_PSGO,LS_DESC_TRPL,LS_DESC_PSGO,LS_DESC_TCOS,MDL,CLS,TP_ICAO,PMD,NR_ASTS,NM_FBRC,PSSO,Municipio,Regiao


In [35]:
df_municipios['MUNICÍPIO - TOM'] = df_municipios['MUNICÍPIO - TOM'].astype(str).str.strip().str.upper()
df_municipios[df_municipios['MUNICÍPIO - TOM'] == "CURITIBA"]

# df_municipios[df_municipios['MUNICÍPIO - TOM'].str.contains("LENÇÓIS PAULISTAS", na=False)]


,CÓDIGO DO MUNICÍPIO - TOM,CÓDIGO DO MUNICÍPIO - IBGE,MUNICÍPIO - TOM,MUNICÍPIO - IBGE,UF
4321,7535,4106902,CURITIBA,Curitiba,PR


In [23]:
df_ocorrencias.head()

,NR_OCR,NR_FI,OP_PDR,CLAS_OCR,DT_OCR,H_OCR,MCP,UF,RG,DCRI_TP,ICAO,LATD,LONG,TP_ADRM,HIST,MTCL,CATG_ANV,OPOR,TP_OCR,FA_OP,OPCAO,DAN_ANV,ADRM_DSTN,ADRM_ORGM,LS_FTL_TRPL,LS_FTL_PSGO,LS_FTL_TCOS,LS_GV_TRPL,LS_GV_PSGO,LS_GV_TCOS,LS_LV_TRPL,LS_LV_PSGO,LS_LV_TCOS,IL_TRPL,IL_PSGO,LS_DESC_TRPL,LS_DESC_PSGO,LS_DESC_TCOS,MDL,CLS,TP_ICAO,PMD,NR_ASTS,NM_FBRC,PSSO,Municipio,Regiao
0,4499,0002/2013,DDM – ESCOLA DE AVIACAO CIVIL LTDA,Acidente,2013-07-27,19:00:00,PEDRO JUAN CABALLERO,Exterior,NaN,FALHA OU MAU FUNCIONAMENTO DO MOTOR,NaN,"-22,5997","-56,0456",NaN,A AERONAVE TEVE PANE DE MOTOR E APARENTEMENTE REALIZOU UMA AUTO-ROTAÇÃO E EFETUOU UM POUSO DE EMERGENCIA EM UMA ÁREA DE ÁRVORES E ARBUSTOS. O PILOTO SOFREU DANOS GRAVES E ESTÁ HOSPITALIZADO EM UM HOSPITAL DA REGIÃO.OBS.: MENSAGEM DA DINAC (AUTORIDADE DE AVIAÇÃO CIVIL DO PARAGUAI): “DENTRO DE LOS REGISTROS DE LA DINAC NO OBRA NINGUNA SOLICITUD NI AUTORIZACIÓN DE INGRESO O PERMANENCIA DE LA MENCIONADA AERONAVE. EL CASO ESTÁ A CARGO DEL MINISTERIO PÚBLICO DEL PARAGUAY CONTRA EL PILOTO DAVID CARLOS FERREIRA COM LA IMPUTACIÓN DE EXPOSICION A PELIGRO DEL TRAFICO AEREO E INTERVENCIONES PELIGROSAS EM EL TRANSITO AEREO.,PTYGP,TPP,ROSINEI CRISTINA GOMES DA S.CARV. E OUTROS / DDM – ESCOLA DE AVIAÇÃO CIVIL LTDA,SCF-PP,NaN,Voo Privado,NaN,NaN,NaN,0,0,0,1,0,0,0,0,0,0,0,0,0,0,R44,H1P,R44,1088.0,4.0,ROBINSON HELICOPTER,True,NaN,NaN
1,3429,1795/2012,NOAR LINHAS AEREAS S/A,Incidente,2012-09-10,09:30:00,FORA DO BRASIL,Indeterminado,NaN,NaN,NaN,"-28,6175","-17,7528",NaN,Aeronave realizando ferry flight de Recife para Kunovice - Rep Tcheca. Decolou de Las Palmas - GCLP - às 0916 Z com destino à Málaga. Subia para o FL100. Ao cruzar FL085 houve variação de torque e queda de Ng do motor esquerdo. Pilotos cortaram imediatamente o motor e retornaram para GCLP em voo monomotor cujo pouso deu-se às 0944Z.,PRNOA,TPR,NOAR LINHAS AÉREAS S/A,NaN,Subida,Voo Regular,Nenhum,XXXX,XXXX,0,0,0,0,0,0,0,0,0,3,0,0,0,0,L410UVP-E20,L2T,L410,6600.0,21.0,AIRCRAFT INDUSTRIES,True,NaN,NaN
2,3531,2044/2012,TAM,Incidente,2012-10-10,21:10:00,FORA DO BRASIL,Indeterminado,NaN,NaN,NaN,"-49,0269","-2,08083",NaN,"2 FAIL e ENG 2 EGT OVER LIMIT, sendo efetuado o corte do motor #2. Declarada emergência (MAYDAY) e efetuado retorno de voo para LFPG, onde a aeronave pousou OVERWEIGHT, porém sem anormalidades.",PTMVE,TPR,TAM,NaN,Subida,Voo Regular,Nenhum,SBGL,LFPG,0,0,0,0,0,0,0,0,0,16,200,0,0,0,A330-223,L2J,A332,230000.0,243.0,AIRBUS S.A.S.,True,NaN,NaN
3,8213,***,TAM,Incidente Grave,2018-10-31,06:30:00,"MISSIONES,ARG",Exterior,NaN,CORTANTE DE VENTO / TEMPESTADE,Fora de Aeródromo,NaN,NaN,-,"A AERONAVE ESTAVA REALIZANDO UM VOO REGULAR LA8050DE SÃO PAULO (BRASIL) PARA SANTIAGO (CHILE). DUERANTE A FASE DE CRUZEIRO, A AERONVE VOOU ATRAVES DE UMA ÁREA TEMPESTUOSA E ENCONTROU GRANIZO. O VOO FOI DEVIADO PARA O AEROPORTO ALTERNATIVO (SAEZ).",PRMHP,TPR,TAM LINHAS AEREAS S.A,WSTRW,Em rota,Voo Regular,Leve,SCEL,SBGR,0,0,0,0,0,0,0,0,0,6,164,0,0,0,A320-214,L2J,A320,77000.0,184.0,AIRBUS S.A.S.,True,NaN,NaN
4,4086,0918/2013,TAM,Ocorrência Anormal,2013-05-05,20:13:00,FORA DO BRASIL,Indeterminado,NaN,NaN,NaN,"-10,6031","-66,9906",NaN,"A aeronave efetuando o voo JJ8086, etapa SBGR/KMCO, durante cruzeiro apresentou mensagem de ADVISORY ENG 1 OIL QTY e ENG 1 OIL LO PR. Efetuado o corte preventivo do motor #1 pela tripulação e alternado o pouso para SVMI.",PTMVC,TPR,TAM,NaN,Em rota,Voo Regular,Nenhum,KMCO,SBGR,0,0,0,0,0,0,0,0,0,13,210,0,0,0,A330-223,L2J,A332,230000.0,243.0,AIRBUS S.A.S.,True,NaN,NaN
